In [ ]:
##### Required packages
import numpy as np
import os, time

In [ ]:
def read_EventCalc(dir_im, flag_export=False):
	"""
	Read outputs from OpenSHA Event Calculator
	"""
	
	## get filenames in dir_im
	files = os.listdir(dir_im)
	print(files)
	
	## GMMs used
	gm_label = [file[0:file.find('_PGA')] for file in files]
	gm_label = np.unique(gm_label)
	
	## set up empty labels for IM dictionary
	im_in = {}
	eq_rup = {}
	
	## get name for metadata file and mport and format data for storage
	metadata = [file for file in files if 'meta' in file.lower()][0]
	print(metadata)
	with open(dir_im + metadata) as f:
		lines = np.asarray([line.split()[0:4] for line in f])
	[eq_rup.update({str(i[0])+'_'+str(i[1]):{'M':float(i[3]),'r':float(i[2])}}) for i in lines]
	
	## get filenames to import for PGA
	if flag_export is True:
		str_add = ''
	else:
		str_add = '_mean'
	
	files_pga = [file for file in files if 'pga'+str_add+'.txt' in file.lower()] # files for pga
	files_pgv = [file for file in files if 'pgv'+str_add+'.txt' in file.lower()] # files for pgv
	files_im = files_pga + files_pgv
	
	## import and format data for storage
	for file in files_im:
		if 'pga' in file.lower():
			label = 'pga'
		elif 'pgv' in file.lower():
			label = 'pgv'
		
		if flag_export is True:
			mat = np.loadtxt(dir_im+file,unpack=True) # import file
			
			mat_red = mat[2:] # remove columns of source and rupture indices
			mean = np.asarray([mat_red[i] for i in range(len(mat_red)) if np.mod(i,3) == 0]) # retrieve ln_mean values
			sig_total = np.asarray([mat_red[i] for i in range(len(mat_red)) if np.mod(i,3) == 1])[0][0] # retrieve total sigma
			sig_inter = np.asarray([mat_red[i] for i in range(len(mat_red)) if np.mod(i,3) == 2])[0][0] # retrieve inter sigma
			sig_intra = (sig_total**2 - sig_inter**2)**0.5 # calculate intra sigma
			
			im_in.update({label+'_mean':mean,
							label+'_sig_total':sig_total,
							label+'_sig_intra':sig_inter,
							label+'_sig_inter':sig_intra})
							
			np.savetxt(dir_im+file[0:file.find('.txt')]+'_mean.txt',np.transpose(mean),fmt='%6.4f')
			np.savetxt(dir_im+file[0:file.find('.txt')]+'_sigma.txt',[sig_total,sig_inter,sig_intra],fmt='%1.3f')
			
		else:
			mean = np.loadtxt(dir_im+file)
			sig = np.loadtxt(dir_im+file[0:file.find('_mean.txt')]+'_sigma.txt')
			
			sig_total = sig[0]
			sig_inter = sig[1]
			sig_intra = sig[2]
			
			im_in.update({label+'_mean':mean,
							label+'_sig_total':sig_total,
							label+'_sig_intra':sig_inter,
							label+'_sig_inter':sig_intra})
	
	if im_in is None:
		print('im_in is empty')
	
	##
	return im_in, eq_rup

In [ ]:
nsite = '100'
dir_im = 'D:\\eventcalc_'+nsite+'\\'

In [ ]:
im_in, eq_rup = read_EventCalc(dir_im)

In [ ]:
eq_rup['0_0']

In [ ]:
im_in.keys()

In [ ]:
im_in['pga_sig_total']

In [ ]:
im_in['pga_mean']